# Jane Street: Neural Network Starter

I try implementing a simple Tensorflow Keras neural network here. Train in Version 17.

**Caution:** The GroupCV method applied in this notebook may cause time leakage problem. Please use [Purged Time-Series CV][1] instead.

[1]: https://www.kaggle.com/marketneutral/purged-time-series-cv-xgboost-optuna

In [1]:
TRAINING = True
USE_FINETUNE = False

In [2]:
import warnings
warnings.filterwarnings('ignore')

#if TRAINING:
#    import cudf
#    import cupy as cp

import os, gc
import pandas as pd
import numpy as np
import xgboost as xgb
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange
from joblib import dump, load

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers.experimental.preprocessing import Normalization


import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args


import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args
import kerastuner as kt

# Preprocessing

In [3]:
TARGET = 'action'
FEATS = ['feature_{}'.format(int(i)) for i in range(130)]
#FIX_FEAT = ['feature_53', 'feature_45', 'feature_69', 'feature_101', 'feature_84', 'feature_121', 'feature_48', 'feature_89', 'feature_93', 'feature_106', 'feature_108', 'feature_99', 'feature_100', 'feature_87', 'feature_120', 'feature_98', 'feature_46', 'feature_128', 'feature_50', 'feature_116', 'feature_71', 'feature_111', 'feature_55', 'feature_49', 'feature_56', 'feature_1', 'feature_51', 'feature_57', 'feature_107', 'feature_41', 'feature_109', 'feature_113', 'feature_123', 'feature_103', 'feature_90', 'feature_91', 'feature_114', 'feature_42', 'feature_54', 'feature_47', 'feature_95', 'feature_96']

In [4]:
if TRAINING:
    print('Loading...')
    train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
    TARGET = 'action'
    FEATS = ['feature_{}'.format(int(i)) for i in range(130)]

    print('Filling...')
    train = train.query('weight > 0').reset_index(drop = True)
    resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']
    y = np.stack([(train[c] > 0.00000).astype('int') for c in resp_cols]).T #Multitarget

    X = train[FEATS].to_numpy()
    wr = train.weight*train['resp'].to_numpy()
    Y = wr*y[:,3]
    X = train[FEATS].to_numpy()
    print('Finish.')

Loading...
Filling...
Finish.


In [5]:
if TRAINING:
    nan_feat = (train[FEATS].isnull().sum()>0)
    NAN_FEAT = nan_feat[nan_feat == True].index
    nan_feat_bool = nan_feat.values
    with open('../input/js-cv-split2/f_mean.npy', 'rb') as f:
        f_mean = np.load(f)
    
    mask2 = np.isnan(X[:,nan_feat_bool]).astype(int)
    X = np.nan_to_num(X) + np.isnan(X).astype(int) * f_mean
    del(train)
    _= gc.collect()
    pd.to_pickle(nan_feat_bool, 'nfb.pkl')

In [6]:
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM, Masking, Embedding, Concatenate, Input, Reshape,Flatten, AveragePooling1D
from tensorflow.keras.layers import concatenate, Softmax
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.metrics import AUC
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Lambda
#from tensorflow.keras.layers import merge
from tensorflow.keras.layers import multiply, Reshape, add
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import roc_auc_score
from collections import defaultdict
from tqdm import tqdm
from tqdm import trange
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
import warnings
warnings.filterwarnings('ignore')

#if TRAINING:
#    import cudf
#    import cupy as cp

import os, gc
import pandas as pd
import numpy as np
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
from joblib import dump, load

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation, Masking, LSTM, Multiply, Add, Flatten
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers import multiply, Reshape, add
from tensorflow.keras.metrics import AUC
from tensorflow.keras.activations import sigmoid

import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args


import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args
import kerastuner as kt



def create_autoencoder(input_dim,output_dim,noise=0.1):
    i = Input(130)
    mask = Input(130)
    encoded = BatchNormalization()(i)
    encoded = GaussianNoise(noise)(encoded)
    
    encoded = Dense(96, activation = 'elu')(encoded)
    encoded = Dense(64,activation='linear')(encoded)
    encoder = Model(inputs=i,outputs=encoded)
    
    return encoder


encoder = create_autoencoder(130, 5, noise=0.1)
encoder.load_weights('../input/js-cv-split2/encoder.hdf5')
encoder.trainable = False

In [8]:
splits = pd.read_pickle('../input/js-cv-split2/cross_validation.pkl')

In [9]:
f = 4

y_train, y_test = y[splits[f][0]], y[splits[f][1]]
X_train, X_test = X[splits[f][0]], X[splits[f][1]]
Y_train, Y_test = Y[splits[f][0]], Y[splits[f][1]]
mask_train, mask_test = mask2[splits[f][0]], mask2[splits[f][1]]
            
encoded_X_train = encoder(X_train).numpy()
encoded_X_test = encoder(X_test).numpy()
            
X_train = np.concatenate((X_train, encoded_X_train, mask_train), axis = -1)
X_test = np.concatenate((X_test, encoded_X_test, mask_test), axis = -1)

wr_train, wr_test = wr[splits[f][0]], wr[splits[f][1]]

del(X, y, encoded_X_train, encoded_X_test, Y, wr, mask_train, mask_test)

_=gc.collect()


# Transformer Starter

In [10]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates


def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)


def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

  # add extra dimensions to add the padding
  # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)




def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)




def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead) 
    but it must be broadcastable for addition.

    Args:
      q: query shape == (..., seq_len_q, depth)
      k: key shape == (..., seq_len_k, depth)
      v: value shape == (..., seq_len_v, depth_v)
      mask: Float tensor with shape broadcastable 
            to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
      output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = Softmax()(scaled_attention_logits)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

In [11]:
def custom_loss(y_true, y_pred):
    return 100 * tf.keras.losses.MSE(y_true,y_pred)

def metrics2(y_true, y_pred):
    return K.sum(y_pred)

def metrics(y_true, y_pred):
    Pi = np.bincount(y_true, y_pred)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / len(Pi))
    u = min(max(t, 0), 6) * np.sum(Pi)
    print('\n', round(u,5))
    return u

In [12]:
from tensorflow.keras.layers import Conv1D, AveragePooling1D, MaxPooling1D, Flatten, Multiply


In [13]:
def transformer_model(input_dim1, window_size, d_model, rate, num_layers, dff, num_heads, output_dim):
    inputs = Input(input_dim1)
    batch_size = tf.shape(inputs)[0]
    #mask = create_look_ahead_mask(seq)
    x = BatchNormalization()(inputs)
    x = Dense(d_model * window_size, use_bias = False)(x)
    x = tf.reshape(x, (batch_size, window_size, d_model))
    
    #x *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    #x += positional_encoding(seq, d_model)
    #x = Dropout(rate)(x)
    
    for i in range(num_layers):
        
        #multihead_attention
        q = Dense(d_model)(x)
        k = Dense(d_model)(x)
        v = Dense(d_model)(x)
        
        q = tf.reshape(q, (batch_size, -1, num_heads, d_model//num_heads))
        q = tf.transpose(q, perm=[0, 2, 1, 3])
        
        k = tf.reshape(k, (batch_size, -1, num_heads, d_model//num_heads))
        k = tf.transpose(k, perm=[0, 2, 1, 3])
        
        v = tf.reshape(v, (batch_size, -1, num_heads, d_model//num_heads))
        v = tf.transpose(v, perm=[0, 2, 1, 3])
        
        attn_out, _ = scaled_dot_product_attention(q, k, v, None)
        attn_out = tf.transpose(attn_out, perm=[0, 2, 1, 3])  
        attn_out = tf.reshape(x, (batch_size, -1, d_model))  
        attn_out = Dense(d_model)(attn_out)  
        attn_out = Dropout(rate)(attn_out)
        attn_out = BatchNormalization()(attn_out + x)
        
        fnn_out = Dense(dff, activation= 'relu')(attn_out)  # (batch_size, seq_len, dff)
        fnn_out = Dense(d_model)(fnn_out)
        fnn_out = Dropout(rate)(fnn_out)
        x = BatchNormalization()(fnn_out + attn_out)
        if i <= 1:
            x = AveragePooling1D(2)(x)
        else:
            x = MaxPooling1D(2)(x)
    
    x = Dense(64)(x)
    x = Flatten()(x)
    #x = BatchNormalization()(x)
    x = Dropout(rate)(x)
    x = Dense(output_dim, activation = 'sigmoid', name = 'label_out')(x)
        
    y2 = tf.math.reduce_mean(x, axis = -1)
    y2 = tf.where(y2 > 0.5, 1, 0)
    y2 = tf.cast(y2, tf.float32)
    wr = Input(1)
    y2 = Multiply(name = 'return_out')([y2,wr])
    
    #lr_schedule = ExponentialDecay(0.01, decay_steps=2000,decay_rate=0.95,staircase=False)
    model = Model(inputs = [inputs, wr], outputs = [x,y2])    
    return model

In [14]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [15]:

from tensorflow.keras.optimizers import Adam

with tpu_strategy.scope():
    model = transformer_model(130 + 64 +88, 16, 128, 0.15, 3, 256, 4, 5)
    model.compile(optimizer = Adam(learning_rate = 0.0005), loss = {'label_out':'binary_crossentropy', 'return_out': custom_loss}, metrics = {'label_out':AUC(name = 'auc'), 'return_out': metrics2})

In [16]:
#hello = JS_utility(validation=([X_test, wr_test], date_test))
model.fit([X_train, wr_train], [y_train, Y_train], validation_data = ([X_test, wr_test], [y_test, Y_test]), epochs = 300, batch_size = 4000, callbacks = [EarlyStopping('val_label_out_auc',mode='max',patience=3,restore_best_weights=True)], verbose = 1)
model.save_weights(f'transformer_no_imputer_{f}.hdf5')

Epoch 1/300
384/384 [==============================] - 34s 56ms/step - loss: 1.5427 - label_out_loss: 0.7231 - return_out_loss: 0.8196 - label_out_auc: 0.5205 - return_out_metrics2: 0.3060 - val_loss: 1.2801 - val_label_out_loss: 0.6895 - val_return_out_loss: 0.5906 - val_label_out_auc: 0.5465 - val_return_out_metrics2: 0.5163
Epoch 2/300
384/384 [==============================] - 13s 35ms/step - loss: 1.4461 - label_out_loss: 0.6920 - return_out_loss: 0.7541 - label_out_auc: 0.5432 - return_out_metrics2: 0.5888 - val_loss: 1.2992 - val_label_out_loss: 0.6889 - val_return_out_loss: 0.6103 - val_label_out_auc: 0.5486 - val_return_out_metrics2: 0.4413
Epoch 3/300
384/384 [==============================] - 13s 35ms/step - loss: 1.3878 - label_out_loss: 0.6899 - return_out_loss: 0.6979 - label_out_auc: 0.5475 - return_out_metrics2: 0.7172 - val_loss: 1.3012 - val_label_out_loss: 0.6885 - val_return_out_loss: 0.6127 - val_label_out_auc: 0.5507 - val_return_out_metrics2: 0.4846
Epoch 4/300
3

In [17]:
with tpu_strategy.scope():
    model = transformer_model(130 + 64 +88, 16, 128, 0.2, 3, 256, 4, 5)
    model.compile(optimizer = Adam(learning_rate = 0.0005), loss = {'label_out':'binary_crossentropy', 'return_out': custom_loss}, metrics = {'label_out':AUC(name = 'auc'), 'return_out': metrics2})

In [18]:
model.fit([X_train, wr_train], [y_train, Y_train], validation_data = ([X_test, wr_test], [y_test, Y_test]), epochs = 300, batch_size = 4000, callbacks = [EarlyStopping('val_label_out_auc',mode='max',patience=10,restore_best_weights=True)], verbose = 1)
model.save_weights(f'transformer_no_imputer_{f}2.hdf5')

Epoch 1/300
384/384 [==============================] - 34s 57ms/step - loss: 1.5080 - label_out_loss: 0.7285 - return_out_loss: 0.7795 - label_out_auc: 0.5193 - return_out_metrics2: 0.3371 - val_loss: 1.2861 - val_label_out_loss: 0.6896 - val_return_out_loss: 0.5965 - val_label_out_auc: 0.5451 - val_return_out_metrics2: 0.5027
Epoch 2/300
384/384 [==============================] - 13s 35ms/step - loss: 1.3938 - label_out_loss: 0.6930 - return_out_loss: 0.7008 - label_out_auc: 0.5412 - return_out_metrics2: 0.5820 - val_loss: 1.2672 - val_label_out_loss: 0.6890 - val_return_out_loss: 0.5782 - val_label_out_auc: 0.5479 - val_return_out_metrics2: 0.5696
Epoch 3/300
384/384 [==============================] - 13s 35ms/step - loss: 1.4555 - label_out_loss: 0.6904 - return_out_loss: 0.7651 - label_out_auc: 0.5461 - return_out_metrics2: 0.6298 - val_loss: 1.2841 - val_label_out_loss: 0.6883 - val_return_out_loss: 0.5958 - val_label_out_auc: 0.5514 - val_return_out_metrics2: 0.5181
Epoch 4/300
3

In [19]:
with tpu_strategy.scope():
    model = transformer_model(130 + 64 +88, 16, 128, 0.25, 3, 256, 4, 5)
    model.compile(optimizer = Adam(learning_rate = 0.0005), loss = {'label_out':'binary_crossentropy', 'return_out': custom_loss}, metrics = {'label_out':AUC(name = 'auc'), 'return_out': metrics2})

In [20]:
model.fit([X_train, wr_train], [y_train, Y_train], validation_data = ([X_test, wr_test], [y_test, Y_test]), epochs = 300, batch_size = 4000, callbacks = [EarlyStopping('val_label_out_auc',mode='max',patience=10,restore_best_weights=True)], verbose = 1)
model.save_weights(f'transformer_no_imputer_{f}3.hdf5')

Epoch 1/300
384/384 [==============================] - 34s 57ms/step - loss: 1.4964 - label_out_loss: 0.7315 - return_out_loss: 0.7649 - label_out_auc: 0.5186 - return_out_metrics2: 0.2644 - val_loss: 1.2748 - val_label_out_loss: 0.6899 - val_return_out_loss: 0.5850 - val_label_out_auc: 0.5444 - val_return_out_metrics2: 0.5106
Epoch 2/300
384/384 [==============================] - 14s 35ms/step - loss: 1.4403 - label_out_loss: 0.6937 - return_out_loss: 0.7467 - label_out_auc: 0.5401 - return_out_metrics2: 0.5714 - val_loss: 1.2801 - val_label_out_loss: 0.6889 - val_return_out_loss: 0.5912 - val_label_out_auc: 0.5494 - val_return_out_metrics2: 0.5677
Epoch 3/300
384/384 [==============================] - 14s 35ms/step - loss: 1.4561 - label_out_loss: 0.6904 - return_out_loss: 0.7656 - label_out_auc: 0.5458 - return_out_metrics2: 0.6997 - val_loss: 1.2653 - val_label_out_loss: 0.6885 - val_return_out_loss: 0.5767 - val_label_out_auc: 0.5518 - val_return_out_metrics2: 0.5691
Epoch 4/300
3

In [21]:
with tpu_strategy.scope():
    model = transformer_model(130 + 64 +88, 16, 128, 0.3, 3, 256, 4, 5)
    model.compile(optimizer = Adam(learning_rate = 0.0005), loss = {'label_out':'binary_crossentropy', 'return_out': custom_loss}, metrics = {'label_out':AUC(name = 'auc'), 'return_out': metrics2})

In [22]:
model.fit([X_train, wr_train], [y_train, Y_train], validation_data = ([X_test, wr_test], [y_test, Y_test]), epochs = 300, batch_size = 4000, callbacks = [EarlyStopping('val_label_out_auc',mode='max',patience=10,restore_best_weights=True)], verbose = 1)
model.save_weights(f'transformer_no_imputer_{f}4.hdf5')

Epoch 1/300
384/384 [==============================] - 34s 56ms/step - loss: 1.4876 - label_out_loss: 0.7384 - return_out_loss: 0.7492 - label_out_auc: 0.5165 - return_out_metrics2: 0.2896 - val_loss: 1.3007 - val_label_out_loss: 0.6902 - val_return_out_loss: 0.6105 - val_label_out_auc: 0.5434 - val_return_out_metrics2: 0.4268
Epoch 2/300
384/384 [==============================] - 13s 35ms/step - loss: 1.4606 - label_out_loss: 0.6951 - return_out_loss: 0.7655 - label_out_auc: 0.5386 - return_out_metrics2: 0.5393 - val_loss: 1.2437 - val_label_out_loss: 0.6896 - val_return_out_loss: 0.5541 - val_label_out_auc: 0.5461 - val_return_out_metrics2: 0.6037
Epoch 3/300
384/384 [==============================] - 14s 35ms/step - loss: 1.4062 - label_out_loss: 0.6911 - return_out_loss: 0.7151 - label_out_auc: 0.5439 - return_out_metrics2: 0.6800 - val_loss: 1.2802 - val_label_out_loss: 0.6885 - val_return_out_loss: 0.5916 - val_label_out_auc: 0.5505 - val_return_out_metrics2: 0.5626
Epoch 4/300
3

In [23]:
with tpu_strategy.scope():
    model = transformer_model(130 + 64 +88, 16, 128, 0.35, 3, 256, 4, 5)
    model.compile(optimizer = Adam(learning_rate = 0.0005), loss = {'label_out':'binary_crossentropy', 'return_out': custom_loss}, metrics = {'label_out':AUC(name = 'auc'), 'return_out': metrics2})

In [24]:
model.fit([X_train, wr_train], [y_train, Y_train], validation_data = ([X_test, wr_test], [y_test, Y_test]), epochs = 300, batch_size = 4000, callbacks = [EarlyStopping('val_label_out_auc',mode='max',patience=10,restore_best_weights=True)], verbose = 1)
model.save_weights(f'transformer_no_imputer_{f}5.hdf5')

Epoch 1/300
384/384 [==============================] - 34s 57ms/step - loss: 1.4968 - label_out_loss: 0.7450 - return_out_loss: 0.7517 - label_out_auc: 0.5144 - return_out_metrics2: 0.2446 - val_loss: 1.2985 - val_label_out_loss: 0.6905 - val_return_out_loss: 0.6080 - val_label_out_auc: 0.5416 - val_return_out_metrics2: 0.3841
Epoch 2/300
384/384 [==============================] - 14s 36ms/step - loss: 1.4432 - label_out_loss: 0.6956 - return_out_loss: 0.7476 - label_out_auc: 0.5364 - return_out_metrics2: 0.5270 - val_loss: 1.2772 - val_label_out_loss: 0.6896 - val_return_out_loss: 0.5876 - val_label_out_auc: 0.5457 - val_return_out_metrics2: 0.5411
Epoch 3/300
384/384 [==============================] - 14s 36ms/step - loss: 1.4421 - label_out_loss: 0.6914 - return_out_loss: 0.7506 - label_out_auc: 0.5416 - return_out_metrics2: 0.6286 - val_loss: 1.2572 - val_label_out_loss: 0.6889 - val_return_out_loss: 0.5683 - val_label_out_auc: 0.5490 - val_return_out_metrics2: 0.5744
Epoch 4/300
3